In [43]:
import pandas as pd
import random

def get_random_row_as_string(dataframe):
    """
    Returns a randomly selected row from a pandas dataframe as a formatted string.
    
    Parameters:
        dataframe (pandas.DataFrame): The input dataframe to select a random row from.
        
    Returns:
        str: A string representation of the randomly selected row.
    """
    # Select a random row index
    random_row_index = random.randint(0, len(dataframe)-1)
    
    # Get the selected row as a pandas series object
    selected_row_series = dataframe.iloc[random_row_index]
    
    # Convert the pandas series to a dictionary
    row_dict = selected_row_series.to_dict()
    
    # Format the dictionary as a string
    row_string = ", ".join([f"{key}: {value}" for key, value in row_dict.items()])
    
    return row_string
# Create a sample dataframe
data = {
    'Name': ['John', 'Alice', 'Bob', 'Emma'],
    'Age': [32, 25, 43, 28],
    'City': ['New York', 'London', 'Paris', 'Sydney']
}
df = pd.DataFrame(data)

# Get a random row as a string
random_row_string = get_random_row_as_string(df)
print(random_row_string)


Name: Bob, Age: 43, City: Paris
